In [ ]:
import sys
from pathlib import Path
from os import getcwd, remove
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent.parent))

In [ ]:
import pandas as pd
import numpy as np
from os.path import join

from datetime import datetime, date, timedelta
import random



from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace

from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta
# pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)


exclusao_pathfile = join(root, 'database', 'casos_confirmados')

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

random.seed(10)

In [ ]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio']
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'] + '/' + municipios['uf']

In [ ]:
total_municipio = pd.read_excel(join(default_input, 'base_seringa.xlsx'))
total_municipio = total_municipio.sort_values(['RS','Municipio'])
total_municipio = total_municipio.rename(columns={'Populacao':'POPULAÇÃO RESIDENTE',
                                'Municipio':'MUNICÍPIO'})
total_municipio = total_municipio.set_index('MUNICÍPIO')
total_municipio

In [ ]:
total_rs = pd.read_excel(join(default_input, 'base_seringa.xlsx'))
total_rs = total_rs.sort_values(['RS','Municipio'])
total_rs = total_rs.groupby('RS')[['Populacao']].sum().reset_index()
total_rs = total_rs.rename(columns={'Populacao':'POPULAÇÃO RESIDENTE'})
total_rs = total_rs.set_index('RS')
total_rs

In [ ]:
list_seringas = [188400,188400,565200,942000,565200,565200,376800,188400,188400]
lista_vacinas = ['BCG','HEPATITE B (RN)','VIP','PENTA','PNEUMO 10','MENINGO C','INFLUEZA (crianças)','HEPATITE A','SCR']
list_caixas = []

filename = f"seringas2.xlsx"
writer = pd.ExcelWriter(join(default_output,'testes', filename),
                    engine='xlsxwriter')

for seringa, vacina, caixa in zip(list_seringas, lista_vacinas, list_caixas):
    
    tabela_base = pd.read_excel(join(default_input, 'base_seringa.xlsx'))
    tabela_base = tabela_base.sort_values(['RS','Municipio'])
    n_testes = seringa
    testes_per_caixas = caixa
    n_caixinhas = n_testes / testes_per_caixas
    
    tabela_base['CAIXAS'] = 1
    partial_total = tabela_base['CAIXAS'].sum()
    
    testes_adicionais = random.choices(tabela_base['Municipio'], weights=tabela_base['Populacao'], k=int(n_caixinhas - partial_total))
    
    tabela_base = tabela_base.join(pd.Series(testes_adicionais, name='CAIXAS_ADICIONAIS').value_counts(), on='Municipio')
    
    tabela_base['CAIXAS'] = tabela_base['CAIXAS'] + tabela_base['CAIXAS_ADICIONAIS'].fillna(0)
    del(tabela_base['CAIXAS_ADICIONAIS'])
    
    tabela_base['RS'] = tabela_base['RS'].astype(int)
    tabela_base['CAIXAS'] = tabela_base['CAIXAS'].astype(int)
    tabela_base['TESTES'] = tabela_base['CAIXAS'] * testes_per_caixas
    tabela_base.loc['TOTAL', ['Populacao', 'CAIXAS', 'TESTES']] = tabela_base[['Populacao', 'CAIXAS', 'TESTES']].sum()
    
    tabela_base['Populacao'] = tabela_base['Populacao'].astype(int)
    tabela_base['CAIXAS'] = tabela_base['CAIXAS'].astype(int)
    tabela_base['TESTES'] = tabela_base['TESTES'].astype(int)
    
    del(tabela_base['CAIXAS'])
    
    #POR MUNICIPIO
    testes_municipios = tabela_base.rename(columns={'Populacao':'POPULAÇÃO RESIDENTE',
                                'TESTES':f"{vacina}",
                                'Municipio':'MUNICÍPIO'})
    testes_municipios = testes_municipios.set_index('MUNICÍPIO')
    total_municipio = total_municipio.join(testes_municipios[f"{vacina}"], on='MUNICÍPIO')

    
    #POR RS
    testes_rs = tabela_base.groupby('RS')[['Populacao', 'TESTES']].sum().reset_index()
    testes_rs.loc['TOTAL', ['Populacao', 'TESTES']] = testes_rs[['Populacao', 'TESTES']].sum()
    testes_rs = testes_rs.rename(columns={'Populacao':'POPULAÇÃO RESIDENTE',
                                'TESTES':f"{vacina}"})
    testes_rs = testes_rs.set_index('RS')
    total_rs = total_rs.join(testes_rs[f"{vacina}"], on='RS')

    
    
total_municipio = total_municipio.reset_index()
total_municipio.to_excel(writer,f"por_municipio", index=False)

total_rs = total_rs.reset_index()
total_rs.to_excel(writer,f"por_rs", index=False)


writer.save()
writer.close()

In [ ]:
# tabela_base = pd.read_excel(join(default_input, 'base_seringa.xlsx'))
# tabela_base = tabela_base.sort_values(['MACRO', 'RS'])
# n_testes = 188400
# testes_per_caixas = 20
# n_caixinhas = n_testes / testes_per_caixas

# tabela_base['CAIXAS'] = 1
# partial_total = tabela_base['CAIXAS'].sum()

# testes_adicionais = random.choices(tabela_base['Municipio'], weights=tabela_base['Populacao'], k=int(n_caixinhas - partial_total))

# tabela_base = tabela_base.join(pd.Series(testes_adicionais, name='CAIXAS_ADICIONAIS').value_counts(), on='Municipio')

# tabela_base['CAIXAS'] = tabela_base['CAIXAS'] + tabela_base['CAIXAS_ADICIONAIS'].fillna(0)
# del(tabela_base['CAIXAS_ADICIONAIS'])

# tabela_base['RS'] = tabela_base['RS'].astype(int)
# tabela_base['CAIXAS'] = tabela_base['CAIXAS'].astype(int)
# tabela_base['TESTES'] = tabela_base['CAIXAS'] * testes_per_caixas
# tabela_base.loc['TOTAL', ['Populacao', 'CAIXAS', 'TESTES']] = tabela_base[['Populacao', 'CAIXAS', 'TESTES']].sum()

# tabela_base['Populacao'] = tabela_base['Populacao'].astype(int)
# tabela_base['CAIXAS'] = tabela_base['CAIXAS'].astype(int)
# tabela_base['TESTES'] = tabela_base['TESTES'].astype(int)

# del(tabela_base['CAIXAS'])
# testes_municipios = tabela_base.rename(columns={'Populacao':'POPULAÇÃO RESIDENTE',
#                             'TESTES':'SERINGAS PREVISTAS',
#                             'Municipio':'MUNICÍPIO'})

# testes_rs = tabela_base.groupby('RS')[['Populacao', 'TESTES']].sum().reset_index()


# testes_rs = testes_rs.rename(columns={'Populacao':'POPULAÇÃO RESIDENTE',
#                             'TESTES':'SERINGAS PREVISTAS'})